<a href="https://colab.research.google.com/github/danlingzhou16/stat390/blob/Danling/Danling_stat390_impute102223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# I am imputing the missing values based on last Thur's discussion with Prof
import pandas as pd
data = pd.read_csv("top_10_gdp_covid_data.csv")

<ipython-input-2-8e4ba57d9f0a>:3: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("top_10_gdp_covid_data.csv")


In [3]:
data.head()

,Unnamed: 0,date,location_key_x,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,lawatlas_mitigation_policy,population,population_male,...,TX.VAL.MRCH.OR.ZS,TX.VAL.MRCH.R1.ZS,TX.VAL.MRCH.R2.ZS,TX.VAL.MRCH.R3.ZS,TX.VAL.MRCH.R4.ZS,TX.VAL.MRCH.R5.ZS,TX.VAL.MRCH.R6.ZS,TX.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TX.VAL.MRCH.XD.WD
0,9896188,2020-01-22,US_AK,0.0,0.0,0.0,0.0,0.0,733391.0,424916.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
1,9896189,2020-01-23,US_AK,0.0,0.0,0.0,0.0,0.0,733391.0,424916.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
2,9896190,2020-01-24,US_AK,0.0,0.0,0.0,0.0,0.0,733391.0,424916.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
3,9896191,2020-01-25,US_AK,0.0,0.0,0.0,0.0,0.0,733391.0,424916.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
4,9896192,2020-01-26,US_AK,0.0,0.0,0.0,0.0,0.0,733391.0,424916.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797


In [4]:
# finding the missing values
data.isna().sum().loc[data.isna().sum() > 0]

new_confirmed                             90
new_deceased                            9859
cumulative_confirmed                       1
cumulative_deceased                     9866
lawatlas_mitigation_policy             88920
population_male                         1936
population_female                       1936
life_expectancy                         9389
mobility_workplaces                     9874
search_trends_infection                43102
search_trends_pain                     43102
school_closing                         43839
workplace_closing                      43838
cancel_public_events                   43838
restrictions_on_gatherings             43838
public_transport_closing               43867
stay_at_home_requirements              43838
restrictions_on_internal_movement      43852
international_travel_controls          43860
income_support                         43934
debt_relief                            43956
public_information_campaigns           43847
testing_po

What to do with each missing data?

To drop:

- `new_confirmed` This is the target variable. This is no point of imputing it. Drop the rows with missing values.
- `lawatlas_mitigation_policy`: to much missing
- `cumulative_confirmed` this is highly correlated with the target variable
- `search_trends_infection`, `search_trends_pain`, `SE.PRE.DURS`,  too much missing
- `school_closing`, `workplace_closing`, `cancel_public_events`, `restrictions_on_gatherings`,`public_transport_closing`,`stay_at_home_requirement`, `restrictions_on_internal_movement`, `international_travel_controls`, `income_support`, `debt_relief`, `public_information_campaigns`, `testing_policy`, `contact_tracing`, `facial_coverings`, `vaccination_policy`, `stringency_index` too much missing and no further explaination from the data source what label encoding means
- `place_id`, `subregion1_code`, `subregion1_name`, `subregion2_code`,
`subregion2_name`: data unrelated to the prediction. We already have location key.


To impute with consideration of time
- `new_deceased` (grouping with cumulative)
- `mobility_workplaces`
- `new_persons_fully_vaccinated` (grouping with cumulative)

To impute (cumulative)
- `cumulative_deceased`
- `cumulative_persons_fully_vaccinated`

To impute without consideration of time
- `population_male`
- `population_female`
- `life_expectancy`
- `AG.LND.CROP.ZS`
- `SP.RUR.TOTL.ZG`



In [5]:
# to drop unecessary columns
data.drop(columns = ['lawatlas_mitigation_policy', 'cumulative_confirmed', 'search_trends_infection','search_trends_pain', 'SE.PRE.DURS',
                     'school_closing', 'workplace_closing', 'cancel_public_events', 'restrictions_on_gatherings', 'public_transport_closing',
                     'stay_at_home_requirements', 'restrictions_on_internal_movement', 'international_travel_controls', 'income_support', 'debt_relief',
                     'public_information_campaigns', 'testing_policy', 'contact_tracing', 'facial_coverings', 'vaccination_policy', 'stringency_index',
                     'place_id', 'subregion1_code', 'subregion1_name', 'subregion2_name', 'subregion2_code', 'Unnamed: 0'], inplace = True)

In [6]:
# drop the rows with missing target variable
data = data.loc[data.new_confirmed.notna()]
data.isna().sum().loc[data.isna().sum() > 0]

new_deceased                            9844
cumulative_deceased                     9855
population_male                         1936
population_female                       1936
life_expectancy                         9307
mobility_workplaces                     9831
new_persons_fully_vaccinated           42963
cumulative_persons_fully_vaccinated    43649
AG.LND.CROP.ZS                           987
SP.RUR.TOTL.ZG                           987
dtype: int64

In [7]:
# some additional useless cols that are not missing:
for col in data.columns:
  print(col)

date
location_key_x
new_confirmed
new_deceased
cumulative_deceased
population
population_male
population_female
latitude
longitude
area_sq_km
life_expectancy
mobility_workplaces
wikidata_id
country_code
country_name
iso_3166_1_alpha_2
iso_3166_1_alpha_3
aggregation_level
new_persons_fully_vaccinated
cumulative_persons_fully_vaccinated
location_key_y
gdp_usd
gdp_per_capita_usd
location_key
AG.LND.AGRI.K2
AG.LND.AGRI.ZS
AG.LND.ARBL.HA
AG.LND.ARBL.HA.PC
AG.LND.ARBL.ZS
AG.LND.CROP.ZS
AG.LND.FRST.K2
AG.LND.FRST.ZS
AG.LND.TOTL.K2
AG.PRD.CROP.XD
AG.PRD.FOOD.XD
AG.PRD.LVSK.XD
AG.SRF.TOTL.K2
BX.KLT.DINV.CD.WD
EG.ELC.ACCS.RU.ZS
EG.ELC.ACCS.UR.ZS
EG.ELC.ACCS.ZS
EG.ELC.RNEW.ZS
EG.FEC.RNEW.ZS
EN.ATM.CO2E.GF.KT
EN.ATM.CO2E.GF.ZS
EN.ATM.CO2E.KT
EN.ATM.CO2E.LF.KT
EN.ATM.CO2E.LF.ZS
EN.ATM.CO2E.PC
EN.ATM.CO2E.SF.KT
EN.ATM.CO2E.SF.ZS
EN.ATM.PM25.MC.M3
EN.ATM.PM25.MC.T1.ZS
EN.ATM.PM25.MC.T2.ZS
EN.ATM.PM25.MC.T3.ZS
EN.ATM.PM25.MC.ZS
EN.BIR.THRD.NO
EN.FSH.THRD.NO
EN.HPT.THRD.NO
EN.MAM.THRD.NO
EN.POP.DNST
ER

In [8]:
useless_cols = ['wikidata_id','location_key_y', 'iso_3166_1_alpha_2', 'iso_3166_1_alpha_3', 'country_name', 'location_key']

In [9]:
data.drop(columns = useless_cols, inplace = True)

In [10]:
# imputing cumulative columns
# make sure date is ordered
data_grouped = data.groupby(['location_key_x'], group_keys=False)
data_reordered_date = data_grouped.apply(lambda x: x.sort_values('date', ascending=True))

In [11]:
data_sel_col = data_reordered_date[['location_key_x','date','cumulative_deceased','new_deceased','cumulative_persons_fully_vaccinated','new_persons_fully_vaccinated']]
data_sel_col_grouped = data_sel_col.groupby(['location_key_x'], group_keys=False)

In [12]:
# two data only grows bigger functions for cumulative deceased and cumulative vaccinated
def data_only_grows_bigger_deceased(df):
  deceased_changed = 0
  for i in df.index:
    try:
      if df.loc[i,'cumulative_deceased'] < df.loc[i-1,'cumulative_deceased']:
        df.loc[i,'cumulative_deceased'] = df.loc[i-1,'cumulative_deceased']
        deceased_changed = deceased_changed + 1
    except:
      pass
  print(deceased_changed, 'rows have been changed')
  return df


def data_only_grows_bigger_vaccinated(df):
  vaccine_changed = 0
  for i in df.index:
    try:
      if df.loc[i,'cumulative_persons_fully_vaccinated'] < df.loc[i-1,'cumulative_persons_fully_vaccinated']:
        df.loc[i,'cumulative_persons_fully_vaccinated'] = df.loc[i-1,'cumulative_persons_fully_vaccinated']
        vaccine_changed = vaccine_changed + 1
    except:
      pass
  print(vaccine_changed, 'rows have been changed')
  return df


In [13]:
data_sel_col_deceased_changed = data_sel_col_grouped.apply(data_only_grows_bigger_deceased)

2 rows have been changed
17 rows have been changed
3 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
2 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
2 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed

In [14]:
# do the same for vaccination data
data_sel_col_deceased_changed_grouped = data_sel_col_deceased_changed.groupby(['location_key_x'], group_keys=False)
df_modifying = data_sel_col_deceased_changed_grouped.apply(data_only_grows_bigger_vaccinated)

134 rows have been changed
3 rows have been changed
2 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
14 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been chang

In [15]:
#let's impute out the missing values for these two cumulative columns
df_modifying_grouped = df_modifying.groupby(['location_key_x'], group_keys=False)
# ffill the values
df_modifying[['cumulative_deceased', 'cumulative_persons_fully_vaccinated']] = df_modifying_grouped[['cumulative_deceased', 'cumulative_persons_fully_vaccinated']].ffill()
# the rest of the missing values should be 0
df_modifying[['cumulative_deceased', 'cumulative_persons_fully_vaccinated']] = df_modifying[['cumulative_deceased', 'cumulative_persons_fully_vaccinated']].fillna(0)

In [16]:
# check the missing values
df_modifying.isna().sum()

location_key_x                             0
date                                       0
cumulative_deceased                        0
new_deceased                            9844
cumulative_persons_fully_vaccinated        0
new_persons_fully_vaccinated           42963
dtype: int64

In [17]:
# making sure that the data is always going up again (this is only a problem for vaccine data)
df_modifying_grouped = df_modifying.groupby('location_key_x', group_keys=False)
df_modifying2 = df_modifying_grouped.apply(data_only_grows_bigger_vaccinated)

0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed
0 rows have been changed


In [18]:
# calculate new_deceased and new_persons_fully_vaccinated
def calculate_new_deceased_vaccine(df):
  deceased_series1 = df.iloc[1:,2].reset_index(drop=True)
  deceased_series2 = df.iloc[:-1,2].reset_index(drop=True)
  new_deceased = deceased_series1 - deceased_series2
  new_deceased = pd.concat([pd.Series([0]), new_deceased], ignore_index=True)
  new_deceased.index = df.index
  df.new_deceased = new_deceased
  vac_series1 = df.iloc[1:,4].reset_index(drop=True)
  vac_series2 = df.iloc[:-1,4].reset_index(drop=True)
  new_vac = vac_series1 - vac_series2
  new_vac = pd.concat([pd.Series([0]), new_vac], ignore_index=True)
  new_vac.index = df.index
  df.new_persons_fully_vaccinated = new_vac
  return df

In [19]:
# group the dataset again to apply this function
df_modifying2_grouped = df_modifying2.groupby('location_key_x', group_keys=False)
df_modifying3 = df_modifying2_grouped.apply(calculate_new_deceased_vaccine)

In [20]:
df_modifying3.isna().sum()
# this dataframe have all deceased and vaccine values filled!

location_key_x                         0
date                                   0
cumulative_deceased                    0
new_deceased                           0
cumulative_persons_fully_vaccinated    0
new_persons_fully_vaccinated           0
dtype: int64

In [21]:
# now put this slice back to the dataset
data_reordered_date[['cumulative_deceased', 'new_deceased','cumulative_persons_fully_vaccinated', 'new_persons_fully_vaccinated']] = df_modifying3[['cumulative_deceased', 'new_deceased','cumulative_persons_fully_vaccinated', 'new_persons_fully_vaccinated']]

In [22]:
data_reordered_date.isna().sum().loc[data_reordered_date.isna().sum() > 0]

population_male        1936
population_female      1936
life_expectancy        9307
mobility_workplaces    9831
AG.LND.CROP.ZS          987
SP.RUR.TOTL.ZG          987
dtype: int64

In [23]:
# split train and test to avoid data leakage
train = data_reordered_date.loc[data.date < '2022-01-01']
train.shape

(70829, 166)

In [24]:
test = data_reordered_date.loc[data.date >= '2022-01-01']
test.shape

(26265, 166)

In [25]:
# to make sure that there is no row left
train.shape[0]+test.shape[0] == data.shape[0]

True

In [26]:
# to remove the target variable in imputing
X_train = train.drop(columns = 'new_confirmed')
y_train = train.new_confirmed
X_test = test.drop(columns = 'new_confirmed')
y_test = test.new_confirmed

In [27]:
X_train.isna().sum().loc[X_train.isna().sum() > 0]

population_male        1420
population_female      1420
life_expectancy        6721
mobility_workplaces    7499
AG.LND.CROP.ZS          731
SP.RUR.TOTL.ZG          731
dtype: int64

In [28]:
X_test.isna().sum().loc[X_test.isna().sum() > 0]

population_male         516
population_female       516
life_expectancy        2586
mobility_workplaces    2332
AG.LND.CROP.ZS          256
SP.RUR.TOTL.ZG          256
dtype: int64

In [29]:
# libraries for scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

In [30]:
# select numerical rows
X_train_numerical = X_train.select_dtypes(include='number')
X_test_numerical = X_test.select_dtypes(include='number')

In [31]:
# feature scaling
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(data = scaler.fit_transform(X_train_numerical), index = X_train_numerical.index, columns = X_train_numerical.columns)
X_test_scaled = pd.DataFrame(data = scaler.transform(X_test_numerical), index = X_test_numerical.index, columns = X_test_numerical.columns)

In [32]:
# imputing missing values
imputer = KNNImputer(n_neighbors=3)
X_train_imputed = pd.DataFrame(data = imputer.fit_transform(X_train_scaled), index = X_train_numerical.index, columns = X_train_numerical.columns)

In [36]:
# scale back the training data
X_train_complete = pd.DataFrame(data = scaler.inverse_transform(X_train_imputed), index = X_train_numerical.index, columns = X_train_numerical.columns)

In [42]:
X_train_complete

,new_deceased,cumulative_deceased,population,population_male,population_female,latitude,longitude,area_sq_km,life_expectancy,mobility_workplaces,...,TX.VAL.MRCH.OR.ZS,TX.VAL.MRCH.R1.ZS,TX.VAL.MRCH.R2.ZS,TX.VAL.MRCH.R3.ZS,TX.VAL.MRCH.R4.ZS,TX.VAL.MRCH.R5.ZS,TX.VAL.MRCH.R6.ZS,TX.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TX.VAL.MRCH.XD.WD
0,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,1.333333,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
1,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,1.333333,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
2,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,1.333333,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
3,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,1.333333,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
4,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,1717856.0,78.0,1.333333,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96918,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,82.4,-72.000000,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
96919,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,82.4,-70.000000,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
96920,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,82.4,-47.000000,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
96921,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,2527013.0,82.4,-47.000000,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005


In [34]:
X_test_imputed = pd.DataFrame(data = imputer.transform(X_test_scaled), index = X_test_numerical.index, columns = X_test_numerical.columns)

In [52]:
# recover the original scale of X_train_imputed and X_test_imputed

X_test_complete = pd.DataFrame(data = scaler.inverse_transform(X_test_imputed), index = X_test_numerical.index, columns = X_test_numerical.columns)

In [61]:
# adding back the non-numerical columns
X_train_completed = X_train.select_dtypes(exclude='number').merge(X_train_complete, left_index=True, right_index=True)
X_test_completed = X_test.select_dtypes(exclude='number').merge(X_test_complete, left_index=True, right_index=True)

In [62]:
X_train_completed

,date,location_key_x,country_code,new_deceased,cumulative_deceased,population,population_male,population_female,latitude,longitude,...,TX.VAL.MRCH.OR.ZS,TX.VAL.MRCH.R1.ZS,TX.VAL.MRCH.R2.ZS,TX.VAL.MRCH.R3.ZS,TX.VAL.MRCH.R4.ZS,TX.VAL.MRCH.R5.ZS,TX.VAL.MRCH.R6.ZS,TX.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TX.VAL.MRCH.XD.WD
0,2020-01-22,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
1,2020-01-23,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
2,2020-01-24,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
3,2020-01-25,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
4,2020-01-26,US_AK,US,0.0,0.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96918,2021-12-27,AU_WA,AU,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
96919,2021-12-28,AU_WA,AU,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
96920,2021-12-29,AU_WA,AU,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
96921,2021-12-30,AU_WA,AU,0.0,9.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005


In [63]:
X_test_completed

,date,location_key_x,country_code,new_deceased,cumulative_deceased,population,population_male,population_female,latitude,longitude,...,TX.VAL.MRCH.OR.ZS,TX.VAL.MRCH.R1.ZS,TX.VAL.MRCH.R2.ZS,TX.VAL.MRCH.R3.ZS,TX.VAL.MRCH.R4.ZS,TX.VAL.MRCH.R5.ZS,TX.VAL.MRCH.R6.ZS,TX.VAL.MRCH.RS.ZS,TX.VAL.MRCH.WL.CD,TX.VAL.MRCH.XD.WD
710,2022-01-01,US_AK,US,0.0,1108.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
711,2022-01-02,US_AK,US,1.0,1109.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
712,2022-01-03,US_AK,US,2.0,1111.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
713,2022-01-04,US_AK,US,2.0,1113.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
714,2022-01-05,US_AK,US,1.0,1114.0,733391.0,424916.0,391925.0,64.0,-150.0,...,39.701735,10.447935,1.398441,23.603533,0.908951,2.398344,0.944532,0.138105,1.664240e+12,213.064797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97179,2022-09-14,AU_WA,AU,0.0,626.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
97180,2022-09-15,AU_WA,AU,0.0,626.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
97181,2022-09-16,AU_WA,AU,0.0,626.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005
97182,2022-09-23,AU_WA,AU,0.0,626.0,2656156.0,1120500.0,1111402.0,-26.0,121.0,...,50.579743,43.125018,0.391859,0.636003,0.412975,5.101246,0.912642,2.018508,2.546920e+11,402.530005


In [64]:
print(X_train_completed.isna().sum().sum())
print(X_test_completed.isna().sum().sum())

0
0


In [66]:
X_train_completed.to_csv('covid_x_train.csv', index=False)
X_test_completed.to_csv('covid_x_test.csv', index=False)

In [67]:
y_train = y_train.to_csv('covid_y_train.csv', index=False)
y_test = y_test.to_csv('covid_y_test.csv', index=False)

Ignore below

In [158]:
try_this = df_modifying2.iloc[2700:2800]

In [159]:
try_this

,location_key_x,date,cumulative_deceased,new_deceased,cumulative_persons_fully_vaccinated,new_persons_fully_vaccinated
2700,US_AR,2022-02-24,10381.0,40.0,1612690.0,687.0
2701,US_AR,2022-02-25,10425.0,44.0,1613219.0,529.0
2702,US_AR,2022-02-26,10465.0,40.0,1613620.0,401.0
2703,US_AR,2022-02-27,10497.0,32.0,1613620.0,0.0
2704,US_AR,2022-02-28,10524.0,27.0,1614676.0,1056.0
...,...,...,...,...,...,...
2795,US_AR,2022-05-30,11484.0,3.0,1651589.0,0.0
2796,US_AR,2022-05-31,11484.0,0.0,1651648.0,59.0
2797,US_AR,2022-06-01,11488.0,4.0,1653127.0,1479.0
2798,US_AR,2022-06-02,11492.0,4.0,1653457.0,330.0


In [160]:
try_this.isna().sum()

location_key_x                         0
date                                   0
cumulative_deceased                    0
new_deceased                           0
cumulative_persons_fully_vaccinated    0
new_persons_fully_vaccinated           0
dtype: int64

In [161]:
calculate_new_deceased_vaccine(try_this)

<ipython-input-152-e0b0f10729da>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.new_deceased = new_deceased
<ipython-input-152-e0b0f10729da>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.new_persons_fully_vaccinated = new_vac


,location_key_x,date,cumulative_deceased,new_deceased,cumulative_persons_fully_vaccinated,new_persons_fully_vaccinated
2700,US_AR,2022-02-24,10381.0,0.0,1612690.0,0.0
2701,US_AR,2022-02-25,10425.0,44.0,1613219.0,529.0
2702,US_AR,2022-02-26,10465.0,40.0,1613620.0,401.0
2703,US_AR,2022-02-27,10497.0,32.0,1613620.0,0.0
2704,US_AR,2022-02-28,10524.0,27.0,1614676.0,1056.0
...,...,...,...,...,...,...
2795,US_AR,2022-05-30,11484.0,3.0,1651589.0,0.0
2796,US_AR,2022-05-31,11484.0,0.0,1651648.0,59.0
2797,US_AR,2022-06-01,11488.0,4.0,1653127.0,1479.0
2798,US_AR,2022-06-02,11492.0,4.0,1653457.0,330.0


In [114]:

df_modifying2_grouped = df_modifying2.groupby('location_key_x')
df_modifying2_grouped.new_deceased =

,location_key_x,date,cumulative_deceased,new_deceased,cumulative_persons_fully_vaccinated,new_persons_fully_vaccinated
0,US_AK,2020-01-22,0.0,0.0,0.0,NaN
1,US_AK,2020-01-23,0.0,0.0,0.0,NaN
2,US_AK,2020-01-24,0.0,0.0,0.0,NaN
3,US_AK,2020-01-25,0.0,0.0,0.0,NaN
4,US_AK,2020-01-26,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...
97179,AU_WA,2022-09-14,626.0,0.0,2287125.0,0.0
97180,AU_WA,2022-09-15,626.0,0.0,2287125.0,0.0
97181,AU_WA,2022-09-16,626.0,NaN,2287125.0,0.0
97182,AU_WA,2022-09-23,626.0,NaN,2287125.0,NaN


In [90]:
try_this = data_sel_col.loc[2200:2300,:]
data_only_grows_bigger_vaccinated(try_this)

0 rows have been changed


,location_key_x,date,cumulative_deceased,new_deceased,cumulative_persons_fully_vaccinated,new_persons_fully_vaccinated
2200,US_AR,2020-10-12,1586.0,17.0,NaN,NaN
2201,US_AR,2020-10-13,1611.0,25.0,NaN,NaN
2202,US_AR,2020-10-14,1634.0,23.0,NaN,NaN
2203,US_AR,2020-10-15,1645.0,11.0,NaN,NaN
2204,US_AR,2020-10-16,1665.0,20.0,NaN,NaN
...,...,...,...,...,...,...
2296,US_AR,2021-01-16,4293.0,30.0,NaN,NaN
2297,US_AR,2021-01-17,4311.0,18.0,NaN,NaN
2298,US_AR,2021-01-18,4343.0,32.0,NaN,NaN
2299,US_AR,2021-01-19,4386.0,43.0,21734.0,2953.0


,location_key_x,date,cumulative_deceased,new_deceased,cumulative_persons_fully_vaccinated,new_persons_fully_vaccinated
2200,US_AR,2020-10-12,1586.0,17.0,NaN,NaN
2201,US_AR,2020-10-13,1611.0,25.0,NaN,NaN
2202,US_AR,2020-10-14,1634.0,23.0,NaN,NaN
2203,US_AR,2020-10-15,1645.0,11.0,NaN,NaN
2204,US_AR,2020-10-16,1665.0,20.0,NaN,NaN
...,...,...,...,...,...,...
2296,US_AR,2021-01-16,4293.0,30.0,NaN,NaN
2297,US_AR,2021-01-17,4311.0,18.0,NaN,NaN
2298,US_AR,2021-01-18,4343.0,32.0,NaN,NaN
2299,US_AR,2021-01-19,4386.0,43.0,21734.0,2953.0


In [57]:
data_reordered_date.cumulative_persons_fully_vaccinated

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
97179    2287125.0
97180    2287125.0
97181          NaN
97182          NaN
97183          NaN
Name: cumulative_persons_fully_vaccinated, Length: 97094, dtype: float64

In [ ]:
data.cumulative_persons_fully_vaccinated

In [35]:
# split train and test to avoid data leakage
train = data.loc[data.date < '2022-01-01']
train.shape

(70829, 166)

In [36]:
test = data.loc[data.date >= '2022-01-01']
test.shape

(26265, 166)

In [37]:
# to make sure that there is no row left
train.shape[0]+test.shape[0] == data.shape[0]

True

In [39]:
# to remove the target variable in imputing
X_train = train.drop(columns = 'new_confirmed')
y_train = train.new_confirmed
X_test = test.drop(columns = 'new_confirmed')
y_test = test.new_confirmed